##### Prediction stock returns with linear regression

##### Imports & Settings

In [13]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [7]:
%matplotlib inline

from time import time
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [8]:
sns.set_style('darkgrid')
idx = pd.IndexSlice

In [9]:
YEAR = 252

In [10]:
DATA = Path('./data')
# DATA_STORE = DATA / 'data.h5'
DATA_READ = DATA / 'data.h5'

##### Load Data

In [11]:
with pd.HDFStore(DATA_READ) as store:
    data = (store['model_data']
            .dropna()
            .drop(['open', 'close', 'low', 'high'], axis=1))

In [14]:
data.index.names = ['symbol', 'date']

data = data.drop([c for c in data.columns if 'lag' in c], axis=1)

##### Select Investment Universe

In [16]:
data = data[data.dollar_vol_rank < 100]

In [17]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 102252 entries, ('A', Timestamp('2013-10-15 00:00:00')) to ('ZTS', Timestamp('2017-11-29 00:00:00'))
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   volume                  102252 non-null  float64
 1   dollar_vol              102252 non-null  float64
 2   dollar_vol_1m           102252 non-null  float64
 3   dollar_vol_rank         102252 non-null  float64
 4   rsi                     102252 non-null  float64
 5   bb_high                 102252 non-null  float64
 6   bb_low                  102252 non-null  float64
 7   atr                     102252 non-null  float64
 8   macd                    102252 non-null  float64
 9   return_1d               102252 non-null  float64
 10  return_5d               102252 non-null  float64
 11  return_10d              102252 non-null  float64
 12  return_21d              102252 non-null  float64
 13  r

##### Create Model Data

In [19]:
y = data.filter(like='target')
X = data.drop(y.columns, axis=1)
X = X.drop(['dollar_vol', 'dollar_vol_rank', \
            'volume'], axis=1)

##### Custom MultipleTimeSeriesCV